<a href="https://colab.research.google.com/github/TianyiZhang-zzz/SupplyChain/blob/main/parcel_pick_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import requests
import math
import time
import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from scipy.optimize import minimize

# Read the Wikipedia table (this assumes the data is in an HTML table)
url = "https://en.wikipedia.org/wiki/Table_of_Atlanta_neighborhoods_by_population"
tables = pd.read_html(url)

# For this example we assume that the first table is the one we want.
# (You may need to check which table is correct.)
df = tables[0]

# Inspect the first few rows
df.head()


,Neighborhood,Population (2010),NPU
0,Adair Park,1331,V
1,Adams Park,1763,R
2,Adamsville,2403,H
3,Almond Park,1020,G
4,Ansley Park,2277,E


In [2]:
# Extract the Top-10 Neighborhoods and Geocode Their Coordinates

# First, rename the population column for ease of use (if desired)
df = df.rename(columns={"Population (2010)": "Population"})

# Convert population to integer (remove commas if present)
df["Population"] = df["Population"].astype(str).str.replace(",", "").astype(int)

# Sort by population (largest first) and select the top 10 neighborhoods.
# (If your sample table has less than 10 rows, adjust accordingly.)
top10 = df.sort_values(by="Population", ascending=False).head(10).copy()

# Display the top 10 neighborhoods with their populations
print(top10[["Neighborhood", "Population"]])

# Set up geopy's Nominatim geocoder (ensure you provide a unique user_agent)
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="atlanta_neighborhood_locator")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Define a function to get latitude and longitude for a neighborhood in Atlanta
def get_coordinates(neighborhood):
    # The query format: "Neighborhood, Atlanta, GA"
    location = geocode(f"{neighborhood}, Atlanta, GA")
    if location:
        return location.latitude, location.longitude
    else:
        return (None, None)

# Apply the function to each neighborhood; unzip the returned tuples into separate columns.
top10['lat'], top10['lon'] = zip(*top10['Neighborhood'].apply(get_coordinates))

# Show the resulting DataFrame with coordinates
print(top10[['Neighborhood', 'Population', 'lat', 'lon']])


               Neighborhood  Population
95                  Midtown       16569
51                 Downtown       13411
104         Old Fourth Ward       10505
101          North Buckhead        8270
119              Pine Hills        8033
98   Morningside/Lenox Park        8030
149       Virginia-Highland        7800
66               Grant Park        6771
64             Georgia Tech        6607
80                 Kirkwood        5897
               Neighborhood  Population        lat        lon
95                  Midtown       16569  33.781656 -84.384071
51                 Downtown       13411  33.763819 -84.385607
104         Old Fourth Ward       10505  33.764108 -84.371763
101          North Buckhead        8270  33.877277 -84.366380
119              Pine Hills        8033  33.563652 -84.259534
98   Morningside/Lenox Park        8030  33.805429 -84.356930
149       Virginia-Highland        7800  33.782656 -84.353691
66               Grant Park        6771  33.735862 -84.370932
64

In [3]:
# Find the "Best" Facility Location Using Haversine Distance

import math
from scipy.optimize import minimize

# Define the haversine function that computes the distance (in kilometers) between two lat/lon points.
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    # Convert latitudes and longitudes from degrees to radians.
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    # Haversine formula.
    a = math.sin(dphi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# Define the objective function:
# For a candidate facility location x = [lat, lon], compute the population-weighted sum of haversine distances.
def objective_haversine(x, data):
    lat0, lon0 = x
    total_distance = 0
    # Sum the distance for each neighborhood, weighted by its population.
    for _, row in data.iterrows():
        d = haversine(lat0, lon0, row['lat'], row['lon'])
        total_distance += row['Population'] * d
    return total_distance

# Compute the population-weighted centroid as an initial guess.
total_population = top10['Population'].sum()
init_lat = (top10['Population'] * top10['lat']).sum() / total_population
init_lon = (top10['Population'] * top10['lon']).sum() / total_population
init_guess = [init_lat, init_lon]
print("Initial guess (population-weighted centroid):", init_guess)

# Run the optimization using the Nelder-Mead method.
result = minimize(objective_haversine, init_guess, args=(top10,), method='Nelder-Mead')
facility_haversine = result.x
print("Optimal facility location based on Haversine distance (lat, lon):", facility_haversine)


Initial guess (population-weighted centroid): [33.76310741466528, -84.36164138230447]
Optimal facility location based on Haversine distance (lat, lon): [ 33.77142401 -84.3761364 ]


In [4]:
# Find the "Best" Facility Location Using Driving Distance

import requests
import time

# Function to get driving distance (in kilometers) using OSRM's public API
def get_driving_distance(lat1, lon1, lat2, lon2):
    # OSRM requires coordinates in the format: lon,lat
    url = f"http://router.project-osrm.org/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false"
    try:
        response = requests.get(url)
        data = response.json()
        time.sleep(0.1)  # be polite with the API calls
        if data.get("code") == "Ok":
            # The distance is returned in meters, so convert to kilometers
            distance = data["routes"][0]["distance"] / 1000
            return distance
        else:
            return float('inf')
    except Exception as e:
        print("Error while fetching driving distance:", e)
        return float('inf')

# Define the objective function for driving distance:
# For a candidate facility location x = [lat, lon], compute the population-weighted sum of driving distances.
def objective_driving(x, data):
    lat0, lon0 = x
    total_distance = 0
    # Sum up each neighborhood's (population * driving distance)
    for _, row in data.iterrows():
        d = get_driving_distance(lat0, lon0, row['lat'], row['lon'])
        total_distance += row['Population'] * d
    return total_distance

# Use the same initial guess (population-weighted centroid) from Part 3.
init_guess = [init_lat, init_lon]
print("Initial guess for driving distance optimization:", init_guess)

# Optimize using the Nelder-Mead method.
# Note: Because each evaluation makes several API calls, we limit the number of iterations.
result_driving = minimize(objective_driving, init_guess, args=(top10,), method='Nelder-Mead', options={'maxiter': 50})
facility_driving = result_driving.x

print("Optimal facility location based on driving distance (lat, lon):", facility_driving)


Initial guess for driving distance optimization: [33.76310741466528, -84.36164138230447]
Optimal facility location based on driving distance (lat, lon): [ 33.77619671 -84.37766847]
